In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, TimeDistributed, GlobalAveragePooling2D, Masking, Rescaling, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
import tensorflow as tf

# === Parameters ===
MAX_FRAMES = 20
IMG_SIZE = 224
NUM_CLASSES = 2

def extract_frames(video_path, max_frames=MAX_FRAMES):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames <= 0:
        return np.zeros((max_frames, IMG_SIZE, IMG_SIZE, 3))
    frame_interval = max(1, total_frames // max_frames)
    for i in range(max_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * frame_interval)
        success, frame = cap.read()
        if not success:
            break
        frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        frames.append(frame)
    cap.release()
    while len(frames) < max_frames:
        frames.append(np.zeros((IMG_SIZE, IMG_SIZE, 3)))
    return np.array(frames)

def load_video_data(df, folder):
    X = []
    for vid in tqdm(df['id'], desc="Loading videos"):
        path = os.path.join(folder, f"{vid}.mp4")
        if os.path.exists(path):
            X.append(extract_frames(path))
        else:
            print(f"Missing: {path}")
            X.append(np.zeros((MAX_FRAMES, IMG_SIZE, IMG_SIZE, 3)))
    return np.array(X)

# === Load DataFrames ===
train_df = pd.read_csv('/kaggle/input/nexar-collision-prediction/train.csv')
test_df = pd.read_csv('/kaggle/input/nexar-collision-prediction/test.csv')

train_df['id'] = train_df['id'].astype(str).str.zfill(5)
test_df['id'] = test_df['id'].astype(str).str.zfill(5)

train_dir = '/kaggle/input/nexar-collision-prediction/train/'
test_dir = '/kaggle/input/nexar-collision-prediction/test/'

# === Load video frames ===
X = load_video_data(train_df, train_dir)
X_test = load_video_data(test_df, test_dir)
y = to_categorical(train_df["target"].values)

In [4]:
import numpy as np
import pandas as pd
import os
import cv2
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, TimeDistributed, GlobalAveragePooling2D, Masking, Rescaling, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
import tensorflow as tf

X= np.load("/kaggle/input/train-data/X.npy")
print(X.shape)

(1500, 20, 224, 224, 3)


In [5]:
X_test= np.load("/kaggle/input/test-data/X_test.npy")
print(X_test.shape)

(1344, 20, 224, 224, 3)


In [6]:
import tensorflow as tf

def downsample_frames(X, target_frames=10, target_size=(112, 112)):
    # 1. Reduce temporal frames
    step = X.shape[1] // target_frames
    X_ds = X[:, ::step, :, :, :]  # Temporal downsampling

    # 2. Resize spatially
    # Flatten to (N*T, H, W, C) to apply tf.image.resize
    N, T, H, W, C = X_ds.shape
    X_flat = X_ds.reshape((-1, H, W, C))
    X_resized = tf.image.resize(X_flat, target_size).numpy()

    # Reshape back to (N, T, new_H, new_W, C)
    return X_resized.reshape((N, target_frames, *target_size, C))

# === Apply downsampling ===
X_down = downsample_frames(X, target_frames=10, target_size=(112, 112))
print(X_down.shape)
X_down_test = downsample_frames(X_test, target_frames=10, target_size=(112, 112))
print(X_down_test.shape)

I0000 00:00:1746305801.404644    2321 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


(1500, 10, 112, 112, 3)
(1344, 10, 112, 112, 3)


In [7]:
# === Load DataFrames ===
train_df = pd.read_csv('/kaggle/input/nexar-collision-prediction/train.csv')
test_df = pd.read_csv('/kaggle/input/nexar-collision-prediction/test.csv')

train_df['id'] = train_df['id'].astype(str).str.zfill(5)
test_df['id'] = test_df['id'].astype(str).str.zfill(5)

train_dir = '/kaggle/input/nexar-collision-prediction/train/'
test_dir = '/kaggle/input/nexar-collision-prediction/test/'

# === Load video frames ===
#X = load_video_data(train_df, train_dir)
#X_test = load_video_data(test_df, test_dir)
y = train_df["target"].values

In [8]:
y

array([0, 1, 0, ..., 1, 1, 1])

In [9]:
print(y.shape)

(1500,)


In [10]:
from tensorflow.keras.metrics import AUC

# === Train/Validation split ===
X_train, X_val, y_train, y_val = train_test_split(X_down, y, stratify=y, test_size=0.2, random_state=42)
# === Set constants ===
MAX_FRAMES = 10
IMG_SIZE = 112
NUM_CLASSES = 1
BATCH_SIZE = 4
EPOCHS =10
# === Build model with TimeDistributed ===
def build_timedistributed_model():
    input_tensor = Input(shape=(MAX_FRAMES, IMG_SIZE, IMG_SIZE, 3))

    # Preprocess input
    x = TimeDistributed(Rescaling(scale=1./255))(input_tensor)
    x = TimeDistributed(Lambda(preprocess_input))(x)

    # Base CNN
    base_cnn = EfficientNetB0(weights='imagenet', include_top=False, pooling=None)
    base_cnn.trainable = True  # Optional: freeze CNN

    # TimeDistributed CNN
    x = TimeDistributed(base_cnn)(x)
    x = TimeDistributed(GlobalAveragePooling2D())(x)  # Output: (batch, time, features)

    # LSTM and classification
    x = Masking(mask_value=0.0)(x)
    x = LSTM(256, return_sequences=False)(x)
    x = Dropout(0.2)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    output = Dense(NUM_CLASSES, activation='sigmoid')(x)  # one output with sigmoid
    model = Model(inputs=input_tensor, outputs=output)

    model.compile(
        optimizer=Adam(1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy', AUC(name='auc')]
    )
    return model

model_lstm = build_timedistributed_model()
model_lstm.summary()

# === Train ===
callbacks = [EarlyStopping(patience=3, restore_best_weights=True)]
model_lstm.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=4, callbacks=callbacks)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 10, 112, 112,   │              0 │ -                      │
│                           │ 3)                     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed          │ (None, 10, 112, 112,   │              0 │ input_layer[0][0]      │
│ (TimeDistributed)         │ 3)                     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_1        │ (None, 10, 112, 112,   │              0 │ time_distributed[0][0] │
│ (TimeDistributed)         │ 3)                     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_2        │ (None, 10, 3, 3, 1280) │      4,049,571 │ time_distributed_1[0]… │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_3        │ (None, 10, 1280)       │              0 │ time_distributed_2[0]… │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal (NotEqual)      │ (None, 10, 1280)       │              0 │ time_distributed_3[0]… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ masking (Masking)         │ (None, 10, 1280)       │              0 │ time_distributed_3[0]… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ any (Any)                 │ (None, 10)             │              0 │ not_equal[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, 256)            │      1,573,888 │ masking[0][0],         │
│                           │                        │                │ any[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 256)            │              0 │ lstm[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │         32,896 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 128)            │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 1)              │            129 │ dropout_1[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 5,656,484 (21.58 MB)

 Trainable params: 5,614,461 (21.42 MB)

 Non-trainable params: 42,023 (164.16 KB)

Epoch 1/10


E0000 00:00:1746306084.609206    2321 meta_optimizer.cc:966] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_1/time_distributed_2_1/block2b_drop_19/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1746306110.304636    2377 cuda_dnn.cc:529] Loaded cuDNN version 90300


300/300 ━━━━━━━━━━━━━━━━━━━━ 355s 379ms/step - accuracy: 0.6025 - auc: 0.6411 - loss: 0.6588 - val_accuracy: 0.6700 - val_auc: 0.7138 - val_loss: 0.6276
Epoch 2/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 79s 263ms/step - accuracy: 0.6986 - auc: 0.7593 - loss: 0.5822 - val_accuracy: 0.6233 - val_auc: 0.6974 - val_loss: 0.6416
Epoch 3/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 79s 264ms/step - accuracy: 0.7384 - auc: 0.8250 - loss: 0.5137 - val_accuracy: 0.6400 - val_auc: 0.6955 - val_loss: 0.6316
Epoch 4/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 79s 263ms/step - accuracy: 0.7621 - auc: 0.8472 - loss: 0.4892 - val_accuracy: 0.6367 - val_auc: 0.6831 - val_loss: 0.7008


In [11]:
# === Evaluate on validation set ===
from tensorflow.keras.metrics import AUC

val_loss, val_accuracy, val_auc = model_lstm.evaluate(X_val, y_val, batch_size=4, verbose=1)

# Print results
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Validation AUC: {val_auc:.4f}")  # AUC value


75/75 ━━━━━━━━━━━━━━━━━━━━ 5s 63ms/step - accuracy: 0.6879 - auc: 0.7573 - loss: 0.6178
Validation Accuracy: 0.6700
Validation AUC: 0.7138


In [13]:
# Make predictions on the test data (X_test_down)
predictions = model_lstm.predict(X_down_test, batch_size=4)
predictions

336/336 ━━━━━━━━━━━━━━━━━━━━ 50s 60ms/step


array([[0.39660633],
       [0.37246585],
       [0.34902406],
       ...,
       [0.54793614],
       [0.3672344 ],
       [0.20900367]], dtype=float32)

In [17]:
test_preds= predictions

IndexError: index 1 is out of bounds for axis 1 with size 1

In [16]:
test_df.shape

(1344, 1)

In [18]:
submission = pd.DataFrame({
    'id': test_df['id'],
    'score': predictions.ravel()  # or use predictions.squeeze()
})
submission.to_csv('/kaggle/working/lstm_submission.csv', index=False)
print("✅ Written → lstm_submission.csv")


✅ Written → lstm_submission.csv


In [19]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

def build_3d_cnn_model():
    input_tensor = Input(shape=(10, 112, 112, 3))  # Shape (num_frames, height, width, channels)

    # 3D Convolutions
    x = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(input_tensor)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)
    
    x = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)
    
    x = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)

    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    output = Dense(1, activation='sigmoid')(x)  # Binary output for crash/no crash
    
    model = Model(inputs=input_tensor, outputs=output)
    model.compile(optimizer=Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy', AUC(name='auc')])
    
    return model


In [16]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

def build_gru_model():
    input_tensor = Input(shape=(10, 112, 112, 3))  # Shape (num_frames, height, width, channels)

    # Preprocess frames with a CNN base (same as before)
    x = TimeDistributed(Rescaling(1./255))(input_tensor)
    x = TimeDistributed(Lambda(preprocess_input))(x)
    
    base_model = EfficientNetB0(weights='imagenet', include_top=False, pooling= None)
    base_model.trainable = True  # You can fine-tune later
    
    x = TimeDistributed(base_model)(x)
    x = GRU(128)(x)  # GRU layer
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    output = Dense(1, activation='sigmoid')(x)  # Binary output for crash/no crash
    
    model = Model(inputs=input_tensor, outputs=output)
    model.compile(optimizer=Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy', AUC(name='auc')])
    
    return model


In [11]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

def build_bag_of_frames_model():
    input_tensor = Input(shape=(10, 112, 112, 3))  # Shape (num_frames, height, width, channels)

    # Preprocess each frame
    x = TimeDistributed(Rescaling(1./255))(input_tensor)
    x = TimeDistributed(Lambda(preprocess_input))(x)

    # Use a simple CNN per frame (like EfficientNet)
    base_model = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')
    base_model.trainable = False  # You can fine-tune later
    
    x = TimeDistributed(base_model)(x)  # Process each frame with CNN
    x = GlobalAveragePooling1D()(x)  # Average all frame features

    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    output = Dense(1, activation='sigmoid')(x)  # Binary output for crash/no crash
    
    model = Model(inputs=input_tensor, outputs=output)
    model.compile(optimizer=Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy', AUC(name='auc')])
    
    return model


In [20]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import AUC

# === Train/Validation split ===
X_train, X_val, y_train, y_val = train_test_split(X_down, y, stratify=y, test_size=0.2, random_state=42)
# === Set constants ===
MAX_FRAMES = 10
IMG_SIZE = 112
NUM_CLASSES = 1
BATCH_SIZE = 4
EPOCHS =10
# Assuming you've already defined X_train, y_train, X_val, y_val
callbacks = [EarlyStopping(patience=3, restore_best_weights=True)]

# Choose one of the models above:
model_3DCNN = build_3d_cnn_model()  # for 3D CNN
#model_gru = build_gru_model()  # for GRU
#model_bag = build_bag_of_frames_model()  # for Bag of Frames

model_3DCNN.fit(X_train, y_train, validation_data=(X_val, y_val),
          epochs=10, batch_size=4, callbacks=callbacks)


Epoch 1/10


I0000 00:00:1746307893.941638    2377 service.cc:148] XLA service 0x7b7986901090 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746307893.944455    2377 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


  7/300 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.5877 - auc: 0.3283 - loss: 34.3823  

I0000 00:00:1746307897.154000    2377 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


300/300 ━━━━━━━━━━━━━━━━━━━━ 14s 29ms/step - accuracy: 0.4983 - auc: 0.4724 - loss: 5.8636 - val_accuracy: 0.5767 - val_auc: 0.5981 - val_loss: 0.6886
Epoch 2/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - accuracy: 0.5220 - auc: 0.5688 - loss: 0.6848 - val_accuracy: 0.6033 - val_auc: 0.6793 - val_loss: 0.6570
Epoch 3/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - accuracy: 0.5972 - auc: 0.6507 - loss: 0.6564 - val_accuracy: 0.6467 - val_auc: 0.7055 - val_loss: 0.6316
Epoch 4/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - accuracy: 0.6289 - auc: 0.6866 - loss: 0.6300 - val_accuracy: 0.6267 - val_auc: 0.7176 - val_loss: 0.6387
Epoch 5/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - accuracy: 0.6285 - auc: 0.6888 - loss: 0.6265 - val_accuracy: 0.6167 - val_auc: 0.7225 - val_loss: 0.6338
Epoch 6/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - accuracy: 0.6468 - auc: 0.7155 - loss: 0.6165 - val_accuracy: 0.6467 - val_auc: 0.7182 - val_loss: 0.6194
Epoch 7/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 7s 2

In [ ]:
val_loss, val_accuracy, val_auc = model_gru.evaluate(X_val, y_val, batch_size=4, verbose=1)

# Print results
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Validation AUC: {val_auc:.4f}")  # AUC value

In [14]:
val_loss, val_accuracy, val_auc = model_bag.evaluate(X_val, y_val, batch_size=4, verbose=1)

# Print results
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Validation AUC: {val_auc:.4f}")  # AUC value

75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - accuracy: 0.5016 - auc: 0.5000 - loss: 0.6932
Validation Accuracy: 0.5000
Validation AUC: 0.5000


In [ ]:
# === Evaluate on validation set ===
val_loss, val_accuracy, val_auc = model.evaluate(X_val, y_val, batch_size=4, verbose=1)

# Print results
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Validation AUC: {val_auc:.4f}")  # AUC value
